In [2]:
def print_dict(d):
    for k, v in d.items():
        print(f"\t'{k}': {v}")

def print_dict2(d):
    l = 0
    for detail in d:
        print()
        print(f'item #{l}:')
        l = l + 1

        i = 0
        for k, v in detail.items():
            if i == 0:
                print(f"\t'{k}': {v}")
            else:
                print(f"\t\t'{k}': {v}")

In [1]:
# !pip install git+https://github.com/datapplab/sparsenet

In [3]:
import sparsenet
from sparsenet.core import sparse
import tensorflow as tf

print(tf.__version__)
print(sparsenet.__file__)
print(sparsenet.core.__file__)

2.9.3
e:\code\projects\timbretransfer\midi-ddsp-main\venv\lib\site-packages\sparsenet\__init__.py
e:\code\projects\timbretransfer\midi-ddsp-main\venv\lib\site-packages\sparsenet\core.py


In [4]:
#!pip install tensorflow==2.12.0

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [20]:
import numpy as np
import plotly.graph_objs as go

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

def draw_image(image_array: np.ndarray):
    if image_array.shape != (28, 28):
        raise ValueError("Expected a 28x28 NumPy array.")

    fig = px.imshow(image_array, color_continuous_scale='gray', origin='lower')
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.update_layout(coloraxis_showscale=False)

    pyo.plot(fig)

In [21]:
draw_image(x_train[0])

In [22]:
import gc

tf.keras.backend.clear_session()
gc.collect()

nunits=250
dens=0.3
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  # tf.keras.layers.Dense(nunits, activation="relu"),
  sparse(units=nunits, density=dens, activation="relu"),
  tf.keras.layers.Dense(10, activation='softmax')
])

lr=1e-3
optimizer = tf.keras.optimizers.Nadam(lr =lr)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ns=12000
model.fit(x_train[:ns], y_train[:ns], epochs=5, batch_size=64,
          validation_data=(x_test[:ns], y_test[:ns]))

weight_type used:  1
Epoch 1/5


e:\code\projects\timbretransfer\midi-ddsp-main\venv\lib\site-packages\keras\optimizers\optimizer_v2\nadam.py:78: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



188/188 [==============================] - 5s 17ms/step - loss: 0.7076 - accuracy: 0.8414 - val_loss: 0.3700 - val_accuracy: 0.9036
Epoch 2/5
188/188 [==============================] - 3s 15ms/step - loss: 0.3095 - accuracy: 0.9160 - val_loss: 0.3081 - val_accuracy: 0.9144
Epoch 3/5
188/188 [==============================] - 2s 13ms/step - loss: 0.2536 - accuracy: 0.9308 - val_loss: 0.2835 - val_accuracy: 0.9163
Epoch 4/5
188/188 [==============================] - 3s 14ms/step - loss: 0.2171 - accuracy: 0.9388 - val_loss: 0.2551 - val_accuracy: 0.9253
Epoch 5/5
188/188 [==============================] - 3s 17ms/step - loss: 0.1884 - accuracy: 0.9467 - val_loss: 0.2368 - val_accuracy: 0.9299


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 module_wrapper (ModuleWrapp  (None, 250)              59050     
 er)                                                             
                                                                 
 dense (Dense)               (None, 10)                2510      
                                                                 
Total params: 61,560
Trainable params: 61,560
Non-trainable params: 0
_________________________________________________________________


In [26]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

#model.run_without_synths = True
#model.run_synth_coder_only = True

#converter = tf.lite.TFLiteConverter.from_keras_model(model_vst)

#converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8, tf.lite.OpsSet.TFLITE_BUILTINS,tf.lite.OpsSet.SELECT_TF_OPS ]

print('starting conversion..')
tflite_model = converter.convert()
print('conversion finished')

import pathlib

tflite_model_path=f'simple_usn.tflite'
tflite_model_file = pathlib.Path(tflite_model_path)
tflite_model_file.write_bytes(tflite_model)
print(f'tflite model saved')

starting conversion..


INFO:tensorflow:Assets written to: C:\Users\vovab\AppData\Local\Temp\tmpk773n58g\assets


INFO:tensorflow:Assets written to: C:\Users\vovab\AppData\Local\Temp\tmpk773n58g\assets


conversion finished
tflite model saved


In [27]:
def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)

    input_details = interpreter.get_input_details()
    #print(f"input_details={[detail['name'] for detail in input_details]}")
    print(f"\n\n Inputs (len={len(input_details)}):")
    #print(input_details)
    print_dict2(input_details)

    output_details = interpreter.get_output_details()
    print(f"\n\n Outputs (len={len(output_details)}):")
    print_dict2(output_details)

    return interpreter, input_details, output_details

In [28]:
def run_tflite_inference(interpreter, inputs):

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    def set_inputs_for_tflite(interpreter, interpreter_input):
        interpreter.set_tensor(0, tf.cast(interpreter_input, input_details[0]['dtype']))

    def get_outputs_from_tflite(interpreter):
        out_tensor = interpreter.get_tensor(output_details[0]['index'])
        return out_tensor

    interpreter.allocate_tensors()

    set_inputs_for_tflite(interpreter=interpreter, interpreter_input=inputs)

    interpreter.invoke()

    tflite_synth_params = get_outputs_from_tflite(interpreter)

    return tflite_synth_params

In [29]:
interpreter, input_details, output_details = load_tflite_model(tflite_model_path)



 Inputs (len=1):

item #0:
	'name': serving_default_flatten_input:0
	'index': 0
	'shape': [ 1 28 28]
	'shape_signature': [-1 28 28]
	'dtype': <class 'numpy.float32'>
	'quantization': (0.0, 0)
	'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
	'sparsity_parameters': {}


 Outputs (len=1):

item #0:
	'name': StatefulPartitionedCall:0
	'index': 15
	'shape': [ 1 10]
	'shape_signature': [-1 10]
	'dtype': <class 'numpy.float32'>
	'quantization': (0.0, 0)
	'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
	'sparsity_parameters': {}


In [30]:
example_input = tf.convert_to_tensor(next(iter(x_train), 4))
example_input = tf.expand_dims(example_input, axis=0)

In [31]:
model_output = model(example_input)
outputs = run_tflite_inference(interpreter=interpreter, inputs=example_input)

In [32]:
print(outputs)
print(model_output)

[[1.6304839e-04 1.0071705e-05 1.3168039e-03 1.5584876e-01 5.0415707e-07
  8.4112340e-01 5.0295603e-05 6.7090325e-04 3.6519032e-04 4.5108856e-04]]
tf.Tensor(
[[1.6304823e-04 1.0071715e-05 1.3168034e-03 1.5584862e-01 5.0415753e-07
  8.4112340e-01 5.0295654e-05 6.7090261e-04 3.6518983e-04 4.5108839e-04]], shape=(1, 10), dtype=float32)
